In [1]:
import sys
import os
script_dir = os.getcwd()
sys.path.append(script_dir)

# Utilizando a API do Google Gemini

Já vimos aqui no blog como escrever programas em Python integrados o **ChatGPT**. 
Neste artigo, veremos como fazer o mesmo, porém integrando com o **Gemini**, da Google.

Apenas lembrando que não vamos usar a interface de usuário do ChatGPT. Todo o acesso será feito
por meio do código de um programa Python, usando uma **API** (Interface de Programação de Aplicações) própria do Gemini.

E o que podemos fazer assim? Podemos criar programas que usam o ChatGPT adaptado ao contexto específico da sua aplicação. Por exemplo:
- Você pode dar, ao ChatGPT, acesso a dados específicos do usuário, como documentos ou atividades agendadas.
- Você pode personalizar o ChatGPT para disparar código Python para realizar cálculos complexos ou outras ações.
- O ChatGPT pode fazer resumos, classificar textos, ou fazer outro tipo de raciocínio em linguagem natural.

Enfim, você pode mesclar a capacidade de linguagem e raciocínio do ChatGPT com a capacidade de programação Python tradicional para criar programas inovadores!

Neste documento, você saberá como acessar e utilizar o **Google Gemini** para esse fim.

## 1 - Configurações Iniciais

### Pre requisitos

1 - Python 3.9 ou superior.

2 - Uma instalação de jupyter para executar o notebook.

3 - Possuir a seguinte biblioteca instalada (google-generativeai)

```bash
pip install google-generativeai



O seguinte modulo ira instalar a biblioteca **google-generativeai** caso a mesma  não esteja instalada.

### Obtendo uma chave para sua API

1 - Acesse o seguinte site: [Google API Key.](https://aistudio.google.com/app/apikey?hl=pt-br)

2 - Caso ainda não esteja logado, faça o login utilizando sua conta Google e aceite os termos de serviço.

3 - Clique no botão **Criar chave de API** para criar uma chave para um novo projeto.

4 - Nesse momento será exibido uma mensagem com os termos de serviço, basta clicar em **OK** para prosseguir.

5 - Uma caixa contendo a opção de **Criar uma chave de API em um novo projeto**, Basta clicar para prosseguir.

6 - Parabéns sua chave de API ja foi gerada basta clicar em copiar para acessá-la.

---

OBS: A chave de API é um elemento crítico para acessar os serviços fornecidos pela Google API. Essa chave funciona como uma senha que autentica suas solicitações e garante que você tenha permissão para acessar os recursos.

Por esse motivo, deve-se prestar muita atenção para que a mesma não fique disponível publicamente.

---

### Armazenando uma chave de API

Por esse motivo você vai precisar salvar com o nome GOOGLE_API_KEY em um arquivo `.env` e carregar este arquivo com o módulo `dotenv`.

In [2]:
from dotenv import load_dotenv, find_dotenv
import google.generativeai as gemini
import google.generativeai.client as client

Primeiramente devemos garantir que a chave da API não fique disponível publicamente, para isso devemos criar um arquivo chamado .env

1 - Crie um arquivvo chamado .env

2 - O preencha da seguinte maneira (GOOGLE_API_KEY='Sua chave da API'.
)

3 - Lembrar de substituir o trecho **Sua chave da API** pela sua chave.

# Explicação do Código

Este notebook explica o funcionamento de três funções escritas em Python para manipulação de uma chave API, utilizando a biblioteca `python-dotenv` para lidar com variáveis de ambiente em um arquivo `.env`.

### Explicação das seguintes funções:

**carrega_chave:**
 Objetivo: Esta função carrega a chave API armazenada no arquivo .env e configura a biblioteca gemini com essa chave.

**verifica_chave:**
 Objetivo: Esta função verifica a existência do arquivo .env no sistema.


In [3]:
def carrega_chave(): # Carrega uma chave do arquivo .env
    _ = load_dotenv(find_dotenv())
    chave = os.getenv("GOOGLE_API_KEY")
    gemini.configure(api_key=os.getenv('GOOGLE_API_KEY'))

    print(f"Chave API carregada com sucesso!")
    return chave

def verifica_chave(): # Verifica a existência de um arquivo .env 
    return find_dotenv()


In [4]:
chave = carrega_chave()

Chave API carregada com sucesso!


Parabéns, agora sua chave já está carregada!

## 2 - Utilizando o serviço (Textual)

### Verificando modelos

Primeiramente devemos verificar quais são todos os modelos disponíveis para utilização dsiponibilizados pelo **Google Gemini**.

Para isso basta utilizar o seguinte comando parea receber a lista com os modelos existentes.


In [5]:
modelos = client.get_default_model_client().list_models()

Agora a variável **modelos** contém uma lista contendo todos os modelos existentes.

Para listar apenas os modelos generativos, utilize um loop `for` para filtrar os dados em que `model.supported_generation_methods` seja igual a `'generateContent'` como demonstra o código a seguir.

In [6]:
for model in modelos:
    if 'generateContent' in model.supported_generation_methods:
        print(model.name)


models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash


---

Principalmente podemos trabalhar com os modelos mais atualizados sendo eles:

gemini-1.5-flash - Sendo o melhor custo benefício. **Será utilizado nesta publicação**

gemini-1.5-pro - Sendo o melhor modelo.

Como chegam a essas conclusões estará disponível em uma publicação futura.

### Testes

Caso queira testar a capacidade de todas as versões possíveis, você pode ainda conectado a sua conta entrar em [Google AI Studio](https://aistudio.google.com/) e testar!

Nesse ambiente você pode selecionar e utilizar das mais diferentes versões existentes. 

Valendo também citar que a versão 2 do gemini em formato de **Preview**

### Primeiros passos com o modelo


Ao final desta publicação existirá uma área própria para comparação entre alguns modelos listados nessa lista.


In [7]:
# Carregando o modelo
model = gemini.GenerativeModel('gemini-1.5-flash')

Nesse caso vou solicitar **"faça um resumo em uma linha de:"** a função **statistics.median_high** do python

A fonte desse trecho é da documentação oficial do python: [Documentação do Python: statistics.median_high](https://docs.python.org/pt-br/3/library/statistics.html#statistics.median_high)


In [8]:
# Recebendo nossa primeira mensagem via código
resposta = model.generate_content("faça um resumo em uma linha de: statistics.median_high(data) Retorna a mediana superior de dados numéricos. Se data for vazio, a exceção StatisticsError é levantada. data pode ser uma sequência ou um iterável. A mediana superior sempre é um membro do conjunto de dados. Quando o número de elementos for ímpar, o valor intermediário é retornado. Se houver um número par de elementos, o maior entre os dois valores centrais é retornado.")

Algumas informações da resposta


In [9]:
print(resposta.text) # Resposta textual
print(resposta.usage_metadata) # Informações como número de tokens usados

`statistics.median_high(data)` retorna o maior dos dois valores do meio de um conjunto de dados, ou o valor do meio se o conjunto de dados tiver um número ímpar de elementos. 

prompt_token_count: 102
candidates_token_count: 41
total_token_count: 143



Logo nesse podemos ja realizar a utilização desse modelo dentro de uma aplicação, exemplo:

Imagine que tenhamos um app que recomenda receitas para o usuário baseando-se nos ingredientes possuídos pelo usuário.

In [10]:
# Pegando o input do suario (simulando uma aplicação).
ingredientes = input("Liste os ingredientes que você possui? ")

Nesse exemplo deveremos utilizar um pouco da noção de prompt engineering.
(Atualizar com o link da post falando sobre prompt engineering)   

In [ ]:
# Forma junto com a variavel ingredientes uma entrada adequada para a nossa proposta
texto_final = (f"Com os seguintes ingredientes: {ingredientes}, escreva uma receita completa que utilize esses e apenas esses ingredientes.")

In [ ]:
# Gera a resposta que estamos buscando
resposta = model.generate_content(texto_final)

In [ ]:
print(resposta.text) # Resposta textual
print(resposta.usage_metadata) # Informações como número de tokens usados

## Moqueca de Picanha com Cuscuz Marroquino e Salada Fresca

**Ingredientes:**

* 1 kg de picanha em cubos
* 1 cebola grande picada
* 4 dentes de alho amassados
* 2 colheres de sopa de azeite de oliva
* 1 xícara de tomate picado
* 1 xícara de caldo de carne
* 1/2 xícara de coentro picado
* 1/4 xícara de cebolinha picada
* Sal e pimenta do reino a gosto

**Para o Cuscuz Marroquino:**

* 1 xícara de cuscuz marroquino
* 1 xícara de caldo de legumes fervente
* 1/4 xícara de ervilha fresca ou congelada
* 1/4 xícara de milho verde em grãos
* 1/4 xícara de cebolinha picada

**Para a Salada:**

* 1 xícara de arroz cozido
* 1/2 xícara de macarrão cozido
* 1/2 xícara de tomate picado
* 1/4 xícara de cebola picada
* 1/4 xícara de coentro picado
* 1/4 xícara de cebolinha picada
* 1/4 xícara de azeite de oliva
* Sal e pimenta do reino a gosto

**Preparo:**

**Moqueca:**

1. Tempere a picanha com sal e pimenta do reino.
2. Em uma panela de fundo grosso, aqueça o azeite de oliva e doure a cebola e o 

Como fica perceptível a receita foi impressa de modo completamente geracional, parabéns!

## 3 - Utilizando imagens

Agora vamos utilizar o serviço utilizando como entrada uma imagem.

Primeiramente devemos carregar uma imagem em uma variável, para isso vou utilizar a biblioteca PLT a imagem está armazenada em "Imagens\animal.jpg"  a partir do diretório que esse script está escrito.

In [3]:
import PIL

In [4]:
img = PIL.Image.open(r'Imagens\animal.jpg')
img

AttributeError: module 'PIL' has no attribute 'Image'

A partir desse momento já é possível utilizar essa imagem como entrada para busca de resposta.

In [ ]:
resposta = model.generate_content(img)

print(resposta.text)

This is a capybara. It is the largest rodent in the world and is native to South America. They are semi-aquatic and can be found in a variety of habitats, including grasslands, swamps, and forests. Capybaras are herbivores and their diet consists of grasses, aquatic plants, and fruits. They are social animals and live in groups of up to 100 individuals.


Caso queira-mos enviar uma imagem e um texto para alguma finalidade específica, por exemplo: 

Identificar um animal em uma imagem. (uma capivara no exemplo da imagem citada)

Basta que enviemos uma lista **[A, B]** contendo:

A - Texto de entrada nesse caso "Identifique esse o animal nessa imagem".

B - Variável contendo a imagem.

In [ ]:
resposta = model.generate_content(["Identifique esse o animal nessa imagem", img])

In [ ]:
print(resposta.text)

É uma capivara. 



## 4 - Utilizando o conceito de chat

Quando se pensa em uma interação contínua com um usuário, vale a pena se utilizar da função chat (Que armazena e utiliza o histórico de mensagens trocadas).

In [ ]:
chat = model.start_chat(history=[])
chat # agora o chat está armazenado nesta variável

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-flash',
        generation_config={},
        safety_settings={},
        tools=None,
        system_instruction=None,
        cached_content=None
    ),
    history=[]
)

In [ ]:
resposta = chat.send_message("Escreva uma linha de uma historia sobre o cachorro bernardo")
print(resposta.text)

Bernardo, um São Bernardo gigante com uma mancha branca em forma de coração no peito, sempre pareceu sentir o peso do mundo, especialmente após o desaparecimento da pequena Clara, sua melhor amiga. 



Agora é possível ver todo o histórico de mensagens usando o seguinte comando

In [ ]:
chat.history

[parts {
   text: "Escreva uma linah de uma historia sobre o cachorro bernardo"
 }
 role: "user",
 parts {
   text: "Bernardo, um São Bernardo gigante com uma mancha branca em forma de coração no peito, sempre pareceu sentir o peso do mundo, especialmente após o desaparecimento da pequena Clara, sua melhor amiga. \n"
 }
 role: "model"]

In [ ]:
response = chat.send_message("Troque o nome de bernardo para severino")
print(response.text)

Severino, um São Bernardo gigante com uma mancha branca em forma de coração no peito, sempre pareceu sentir o peso do mundo, especialmente após o desaparecimento da pequena Clara, sua melhor amiga. 




A metodologia acima garante com que haja uma continuidade no envio de mensagens para o modelo, permitindo com que alterações em determinadas partes sejam possíveis. (como demonstrado pela troca do nome do cachorro bernardo).

In [ ]:
for mensagem in chat.history:
  print(f'{mensagem.role}: {mensagem.parts[0].text}')

user: Escreva uma linah de uma historia sobre o cachorro bernardo
model: Bernardo, um São Bernardo gigante com uma mancha branca em forma de coração no peito, sempre pareceu sentir o peso do mundo, especialmente após o desaparecimento da pequena Clara, sua melhor amiga. 

user: Troque o nome de bernardo para severino
model: Severino, um São Bernardo gigante com uma mancha branca em forma de coração no peito, sempre pareceu sentir o peso do mundo, especialmente após o desaparecimento da pequena Clara, sua melhor amiga. 



Parabéns, agora você compreende como o até mesmo o chat funciona, mas vamos é necessário realizar uma comparação entre alguns modelos existentes no **google Gemini**.